In [1]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import copy
client = pymongo.MongoClient('localhost',27017)
start = datetime(2019,12,13)
end = datetime(2020,4,12)

In [2]:
ps = '''广东
北京
海外
山东
江苏
浙江
四川
湖北
上海
河南
河北
福建
安徽
湖南
重庆
辽宁
陕西
广西
黑龙江
江西
山西
天津
内蒙古
云南
吉林
贵州
新疆
甘肃
海南
香港
宁夏
西藏
台湾
青海
澳门'''.split('\n')
ps

['广东',
 '北京',
 '海外',
 '山东',
 '江苏',
 '浙江',
 '四川',
 '湖北',
 '上海',
 '河南',
 '河北',
 '福建',
 '安徽',
 '湖南',
 '重庆',
 '辽宁',
 '陕西',
 '广西',
 '黑龙江',
 '江西',
 '山西',
 '天津',
 '内蒙古',
 '云南',
 '吉林',
 '贵州',
 '新疆',
 '甘肃',
 '海南',
 '香港',
 '宁夏',
 '西藏',
 '台湾',
 '青海',
 '澳门']

In [3]:
py = '''guangdong
beijing
haiwai
shandong
jiangsu
zhejiang
sichuan
hubei
shanghai
henan
hebei
fujian
anhui
hunan
chongqing
liaoning
shanxi
guangxi
heilongjiang
jiangxi
shanxi2
tianjin
neimenggu
yunnan
jilin
guizhou
xinjiang
gansu
hainan
xianggang
ningxia
xizang
taiwan
qinghai
aomen'''.split('\n')
py

['guangdong',
 'beijing',
 'haiwai',
 'shandong',
 'jiangsu',
 'zhejiang',
 'sichuan',
 'hubei',
 'shanghai',
 'henan',
 'hebei',
 'fujian',
 'anhui',
 'hunan',
 'chongqing',
 'liaoning',
 'shanxi',
 'guangxi',
 'heilongjiang',
 'jiangxi',
 'shanxi2',
 'tianjin',
 'neimenggu',
 'yunnan',
 'jilin',
 'guizhou',
 'xinjiang',
 'gansu',
 'hainan',
 'xianggang',
 'ningxia',
 'xizang',
 'taiwan',
 'qinghai',
 'aomen']

In [4]:
pro_df = pd.DataFrame({'c':ps,'e':py})
pro_df

,c,e
0,广东,guangdong
1,北京,beijing
2,海外,haiwai
3,山东,shandong
4,江苏,jiangsu
5,浙江,zhejiang
6,四川,sichuan
7,湖北,hubei
8,上海,shanghai
9,河南,henan


In [5]:
already_pro = client.Emotion2_Count_Db.collection_names()
run_pro = pro_df[~pro_df['e'].isin(already_pro)]
run_pro

,c,e
0,广东,guangdong
1,北京,beijing
2,海外,haiwai
3,山东,shandong
4,江苏,jiangsu
5,浙江,zhejiang
6,四川,sichuan
7,湖北,hubei
8,上海,shanghai
9,河南,henan


In [ ]:
from tqdm import tqdm
co = client.Weibo_Local_Db.comment

for province in run_pro['c'].tolist():
    id_list = [x['_id'] for x in client.Weibo_Local_Db.user.find({'province':province},{})]
    start_one = copy.deepcopy(start)
    insert_co = client.Emotion2_Count_Db[run_pro[run_pro['c'] == province]['e'].tolist()[0]]
    insert_co.ensure_index('datetime',unique=True)
    
    while start_one<end:
        end_one = start_one+timedelta(days=1)
        
        query = {'comment_dt':
                 {'$gt':start_one,
                  '$lte':end_one
                 },
                 'comment_usr_id':
                 {'$in':id_list}
                }
        df = pd.DataFrame(co.find(query))
        data = {}
        data['全部']=len(df)
        for label in tqdm('快乐 乐观 平静 担忧 无助 恐惧 悲伤 愤怒 恐慌 压力 抑郁 焦虑'.split(' ')):
            if label in df.columns:
                data[label]=len(df[label].dropna(how='any'))
            else:
                data[label]=0
        data['datetime'] = start_one
        
        print(province,start_one, end_one, len(df))
        insert_co.insert_one(data)
        start_one += timedelta(days=1)

/home/lowinli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: ensure_index is deprecated. Use create_index instead.
  
100%|██████████| 12/12 [00:00<00:00, 4646.14it/s]


广东 2019-12-13 00:00:00 2019-12-14 00:00:00 169


100%|██████████| 12/12 [00:00<00:00, 637.46it/s]


广东 2019-12-14 00:00:00 2019-12-15 00:00:00 810


100%|██████████| 12/12 [00:00<00:00, 1436.49it/s]


...

江苏 2020-04-09 00:00:00 2020-04-10 00:00:00 1211


100%|██████████| 12/12 [00:00<00:00, 1062.66it/s]


江苏 2020-04-10 00:00:00 2020-04-11 00:00:00 563


100%|██████████| 12/12 [00:00<00:00, 3436.08it/s]


江苏 2020-04-11 00:00:00 2020-04-12 00:00:00 206
